In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
# add project root folder to path to allow import local modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import local modules
from stochastic_models import *
from visualisations import *
from basic_estimator_model import *

In [2]:
np.random.seed(2020)

## Model Parameters

In [28]:
N_train = 2000
N_test = 10000
d = 1
T = 2


In [29]:
lambda_range = (N_train*1e-9 , N_train*1e-3)
alpha_range = (8.3*1e-5, 0.83)
length_scale = np.sort(1/np.sqrt((2*alpha_range[0], 2*alpha_range[1])))

In [30]:
def reconstruct_alpha(length_scale):
    return 1/(2*(length_scale**2))
def reconstruct_lambda(noise_level, N):
    return noise_level/N

## Kernel Construction

In [31]:
#Kernel Construction, doubt on the lambda hyperparameter
kernel = RBF(length_scale= (length_scale[0] + length_scale[1])/2, length_scale_bounds=length_scale) \
        + WhiteKernel(noise_level= (lambda_range[0] + lambda_range[1])/2 , noise_level_bounds=lambda_range)


## Data Generation (Test, Train sets)

In [32]:
s_train = MaxCallStochasticModel(N_train,d,[1/12,11/12])
s_train.generate_samples()
s_test = MaxCallStochasticModel(N_test, d, [1/12,11/12])
s_test.generate_samples()

In [33]:
y_train = s_train.y
X_train = s_train.X
S_train = s_train.S



In [34]:
y_test = s_test.y
X_test = s_test.X
S_test = s_test.S

V_T = y_test
V_0 = s_test.generate_true_V(0)
V_0 = V_0[0][1]

## Data Flattening 

In [35]:
def Flatten_Training_Sample(X , f):
    return np.array([f(x) for x in X])

In [36]:
#Shape of each sample j: [X_j[0,0],... ,X_j[d,0],...,X_j[0,T] , ..., X_j[d,T]]
Flatten_X_1_train = Flatten_Training_Sample(X_train, lambda x : x.T.flatten())
Flatten_X_1_test = Flatten_Training_Sample(X_test, lambda x : x.T.flatten())


#Shape of each sample j: [X_j[0,0],... ,X_j[0,T],...,X_j[d,0] , ..., X_j[d,T]]
Flatten_X_2_train = Flatten_Training_Sample(X_train, lambda x : x.flatten())
Flatten_X_2_test = Flatten_Training_Sample(X_test, lambda x : x.flatten())

In [37]:
Flatten_X_1_train.shape

(2000, 2)

In [38]:
Flatten_X_1_test.shape

(10000, 2)

## Model Fitting

In [39]:
#Model with Flatten_X_1
m_1 = EstimatorModelBase(kernel)
m_1.fit(Flatten_X_1_train,y_train)
fX_1 = m_1._predict_fX(Flatten_X_1_test)

In [15]:
#Model with Flatten_X_2
m_2 = EstimatorModelBase(kernel)
m_2.fit(Flatten_X_2_train, y_train)
fX_2 = m_2._predict_fX(Flatten_X_2_test)

## Optimal Hyperparameters

In [40]:
optimal_hyperparameters_1 = (reconstruct_alpha(np.exp(m_1.kernel.theta[0])),reconstruct_lambda(np.exp(m_1.kernel.theta[1]), N_train))
optimal_hyperparameters_1

(0.00032545828840309794, 0.0005000005)

In [41]:
optimal_hyperparameters_2 = (reconstruct_alpha(np.exp(m_2.kernel.theta[0])), reconstruct_lambda(np.exp(m_2.kernel.theta[1]), N_train))
optimal_hyperparameters_2

(0.00032545828840309794, 0.0005000005)

## Error Calculation

In [42]:
Normalized_Error_T_1 = np.sqrt(1/N_test*np.sum((fX_1-V_T)**2, axis=0))/V_0
## see formula p4 for ||f(X)||2,Q 
## since samples are drawn according to measure - just sum them up

In [43]:
Normalized_Error_T_1

0.12443414183931172

In [44]:
Normalized_Error_T_2 = np.sqrt(1/N_test*np.sum((fX_2-V_T)**2, axis=0))/V_0

ValueError: operands could not be broadcast together with shapes (6000,) (10000,) 

In [21]:
Normalized_Error_T_2

0.12752515249655536

## Reconstruction of V_X (to be done later)

In [22]:
def m_s(x, alpha, betha, d):
    return ((1+2*alpha)**(-d/2))*np.exp(((betha**2+4*alpha*betha - 2*alpha)*np.sum(x**2, axis=0)) / (4*alpha +2))

In [23]:
def w(x, d, T, gamma):
    return ((1-2*gamma)**(d*T))* np.exp(gamma*np.sum(x**2, axis=0))

In [24]:
def M_0(X): 
    M = np.zeros((X.shape[0], T))
    for j in range(len(M)):
        M[j, :]= np.array([m_s(x, optimal_hyperparameters_2[0], 0, d) for x in X[j,:,:].T ])
    return np.prod(M, axis=1)

In [25]:
def M_T(X, kernel):
    M = np.zeros(X.shape[0])
    for j in range(len(M)):
        M[j] = np.prod(kernel.diag(X[j]), axis=0)
    return M

In [26]:
#M_0(X)[j] == M_0(X(j)) in equation (25) for when V_X_t = V_X,0
#M_T(X)[j] == M_T(X(j)) in equation (25) for when V_X_t = V_X,T

In [27]:
K = kernel(Flatten_X_1, Flatten_X_1)

NameError: name 'Flatten_X_1' is not defined

In [ ]:
A = (1/N)*(K + optimal_hyperparameters_1[1])
g = np.linalg.solve(A,y)

In [ ]:
#TODO
def recontruct_V_X (g, w, K, M_0, M_T, N, t):
    if (t == 0):
        
        V_X = (1/N)*(np.sum((M_0*g)/w, axis=0))
    
    if (t == T):
        V_X = (1/N)*(np.sum((M_T*g)/w, axis=0))
    
    return V_X